# ML Trader

In [ ]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

## 1. No need to make 10 transactions a day. Just one is enough
## 2. See the lema gap size on the losing months
## 3. Can I use actual tick value (close) instead of lema for moves?

In [ ]:
%%time
data                = get_date_list(data)
data                = read_data(data)

In [ ]:
%%time
data                = format_tick_time(data)

In [ ]:
sys.exit()

In [ ]:
%%time
data = run_engine(data)

In [ ]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
sys.exit()

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
df_small

In [ ]:
order_side = 'positive'
y = pd.DataFrame(data['df_small'][data['df_small']['pl_type'] == order_side].lema_gap.round(3).value_counts())
y= y.reset_index()
y = y.rename(columns = {'lema_gap':order_side})

order_side = 'negative'
x= pd.DataFrame(data['df_small'][data['df_small']['pl_type'] == order_side].lema_gap.round(3).value_counts())
x = x.reset_index()
x = x.rename(columns = {'lema_gap':order_side})

all = x.merge(y, on = 'index', how = 'outer')
all = all.fillna(0)

all['diff'] = all['positive'] - all['negative']
all = all.sort_values(by = ['diff', 'index'])
all

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [ ]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 8
data['plot_start']  = f"{data['input_year']}-{month}-1"
data['plot_stop']   = f"{data['input_year']}-{month+1}-1"
# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']

plot_graph(data)

In [ ]:
x = 252
data['df_ohlc']['lema_angle'][x-3:x+3]

generate_result_report(data)